In [ ]:
## ------------------------------------------ packages ------------------------------------------ ##
import sys
import cherrypy
import json
import redis
import datetime


## ------------------------------------- initial variables -------------------------------------- ##
REDIS_HOST = 'redis-19224.c293.eu-central-1-1.ec2.cloud.redislabs.com'
REDIS_PORT = 19224
REDIS_USERNAME = 'default'
REDIS_PASSWORD = 'XT6DLlALIOibGPPIrPXZk1cYe6NGca32'

REST_HOST = '0.0.0.0' # localhost
REST_PORT = 8080 # typically an open port

## -------------------------------------- redis connection -------------------------------------- ##
print('Redis connection status:')
try:   
    redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)
    is_connected = redis_client.ping()
    print(f'  -- connection result \u2192 {is_connected}')
except:
    print(f'  -- connection result \u2192 False')
    sys.exit()


## ----------------------------------------- endpoints ----------------------------------------- ##
class Status(object):
    exposed = True

    def GET(self, *path, **query):
        response = json.dumps({
            'status': 'online'
        })
        return response

class Devices(object):
    exposed = True

    def GET(self, *path, **query):

        blt = query.get('blt', None)
        if blt is not None:
            try:
                blt = int(blt)
                if blt < 0 or blt > 100:
                    raise Exception()
            except:
                raise cherrypy.HTTPError(400, 'Bad Request: invalid battery threshold value')
        
        plugged = query.get('plugged', None)
        if plugged is not None:
            try:
                plugged = int(plugged)
                if not plugged in (0, 1):
                    raise Exception()
            except:
                raise cherrypy.HTTPError(400, 'Bad Request: invalid plugged value')
    
        battery_ts_names = [key.decode() for key in redis_client.keys('*:battery')]
        power_ts_names = [key.decode() for key in redis_client.keys('*:power')]
        mac_addresses_on_redis = set()
        for battery_ts_name in battery_ts_names:
            mac_address = battery_ts_name.split(':')[0]
            power_ts_name = mac_address+':power'
            if power_ts_name in power_ts_names:
                mac_addresses_on_redis.add(mac_address)
        
        mac_addresses_to_return = []
        for mac_address in mac_addresses_on_redis:
            battery_ts_name = mac_address+':battery'
            power_ts_name = mac_address+':power'
            last_bettery_level = redis_client.ts().get(battery_ts_name, True)[1]
            last_power_plugged = redis_client.ts().get(power_ts_name, True)[1]
            if (blt is None or last_bettery_level <= blt) and (plugged is None or last_power_plugged == plugged):
                mac_addresses_to_return.append(mac_address)

        response = json.dumps({
            'mac_addresses': mac_addresses_to_return
        })
        return response

class Device(object):
    exposed = True

    def GET(self, *path, **query):

        if len(path) == 1:
            mac_address = path[0]
        else:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address')
        
        start_date = query.get('start_date', None)
        if start_date is not None:
            try:
                start_date = datetime.date.fromisoformat(start_date)
            except:
                raise cherrypy.HTTPError(400, 'Bad Request: wrong format for start date')
        else:
            raise cherrypy.HTTPError(400, 'Bad Request: missing start date')
        
        end_date = query.get('end_date', None)
        if end_date is not None:
            try:
                end_date = datetime.date.fromisoformat(end_date)
            except:
                raise cherrypy.HTTPError(400, 'Bad Request: wrong format for end date')
        else:
            raise cherrypy.HTTPError(400, 'Bad Request: missing end date')
        
        if end_date <= start_date:
            raise cherrypy.HTTPError(400, 'Bad Request: end date smaller or equal than start date')

        battery_ts_name = mac_address+':battery'
        power_ts_name =  mac_address+':power'
        if redis_client.exists(battery_ts_name) == 0 or redis_client.exists(power_ts_name) == 0:
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address')
        
        from_datetime_utc = datetime.datetime.combine(start_date, datetime.time(0, 0))
        from_timestamp_in_ms = int(1000 * from_datetime_utc.timestamp())

        to_datetime_utc = datetime.datetime.combine(end_date, datetime.time(23, 59))
        to_timestamp_in_ms = int(1000 * to_datetime_utc.timestamp())

        battery_data = redis_client.ts().range(battery_ts_name, from_timestamp_in_ms, to_timestamp_in_ms)
        power_data = redis_client.ts().range(power_ts_name, from_timestamp_in_ms, to_timestamp_in_ms)

        timestamps = []
        battery_levels = []
        power_pluggeds = []
        for battery_pair, power_pair in zip(battery_data, power_data):
            timestamp_in_ms_battery, battery_level = battery_pair
            timestamp_in_ms_power, power_plugged = power_pair
            if timestamp_in_ms_battery == timestamp_in_ms_power:
                timestamps.append(timestamp_in_ms_battery)
                battery_levels.append(int(battery_level))
                power_pluggeds.append(int(power_plugged))

        response = json.dumps({
            'mac_address': mac_address,
            'timestamps': timestamps,
            'battery_levels': battery_levels,
            'power_plugged': power_pluggeds
        })
        return response
    
    def DELETE(self, *path, **query):

        if len(path) == 1:
            mac_address = path[0]
        else:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address')
        
        battery_ts_name = mac_address+':battery'
        power_ts_name =  mac_address+':power'
        if redis_client.exists(battery_ts_name) == 0 or redis_client.exists(power_ts_name) == 0:
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address')
        
        redis_client.delete(battery_ts_name)
        redis_client.delete(power_ts_name)
        return json.dumps({})


## ------------------------------------------ setup -------------------------------------------- ##
if __name__ == '__main__':
    conf = {'/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}}
    cherrypy.tree.mount(Status(), '/status', conf)
    cherrypy.tree.mount(Devices(), '/devices', conf)
    cherrypy.tree.mount(Device(), '/device', conf)
    cherrypy.config.update({'server.socket_host': REST_HOST})
    cherrypy.config.update({'server.socket_port': REST_PORT})
    cherrypy.engine.start()
    cherrypy.engine.block()

Redis connection status:
[30/Jan/2024:13:37:03] ENGINE Bus STARTING
[30/Jan/2024:13:37:03] ENGINE Started monitor thread 'Autoreloader'.
[30/Jan/2024:13:37:03] ENGINE Serving on http://0.0.0.0:8080
[30/Jan/2024:13:37:03] ENGINE Bus STARTED
  -- connection result → True
172.3.73.62 - - [30/Jan/2024:13:37:10] "GET /devices?blt=something HTTP/1.1" 400 1814 "" "vscode-restclient"
172.3.73.62 - - [30/Jan/2024:13:37:16] "GET /devices?blt=-1 HTTP/1.1" 400 1763 "" "vscode-restclient"
172.3.70.84 - - [30/Jan/2024:13:37:19] "GET /devices?blt=101 HTTP/1.1" 400 1763 "" "vscode-restclient"
172.3.70.84 - - [30/Jan/2024:13:37:22] "GET /devices?plugged=something HTTP/1.1" 400 1792 "" "vscode-restclient"
172.3.68.226 - - [30/Jan/2024:13:37:25] "GET /devices?plugged=-5 HTTP/1.1" 400 1733 "" "vscode-restclient"
172.3.73.62 - - [30/Jan/2024:13:37:27] "GET /devices?blt=something&plugged=1 HTTP/1.1" 400 1814 "" "vscode-restclient"
172.3.68.226 - - [30/Jan/2024:13:37:36] "GET /devices?blt=99&plugged=somethin

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c66c7ed3-adef-44a1-955f-b8ce9193a2a0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>